<a href="https://colab.research.google.com/github/AlessandroMirone/MBA_UkraineWar_2022/blob/main/mark2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Abstract

IMPORTANT: WHEN THE TEXTUAL PART IS DONE, RE-LINK THE NOTEBOOK:
- import the notebook in colab 
- save it on github : this will overwrite the link to the current version 
- revrite on drive


### Dataset

1) description of dataset 

2) how data are organized (in this toy example and in a real situation) 

steps of the analysis - 1 - loading and preprocessing: 
In a distributed environment (Eg. Hadhoop) data have to be loaded into the distributed storage system. We assume this step is already carried out. Data would be in zip form to save space. 

after the data have been loaded, they are exported to the environments of each node with pd.read.csv(). The data are still saved as zipped file inside each node memory, but now they can be accessed.

we can prepare them for computation using spark by creating an RDD with parallelize. In this step, only english tweets are retained -> rdd = df.parallelize(df.text[df.language = 'english']) 

we can then transform this rdd as to obtain a single array splitted between the nodes, with positions in the array occupied by the texts of each tweet (ie baskets): a map function will take as inputs the 'text' column's values and the pd.Series function, stacking each text content in the positions of the array. The result will be an rdd containing an array of lists split between the different nodes. Each list contain a single element (the full text) 

we can use another map function with split in order to separate the lists into lists of single elements, each list of single elements now occupying the corresponding position in the array.

we can then apply the cleaning functions to the array: each list (every list containing the sequence of words in the corresponding text) will be cleaned and lemmatized. The result is the same, an array of lists (baskets) each containing the tokens present in the relative basket as single elements. We can directly apply the a-priori algorithm to this array.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 53.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ec9ed61ea3f2b554a8f7d77f8e288e4eccb28638807d21645df9ea951ae8931c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 


In [3]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 5.1 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=2442d98b4fcd94edcf2d0f4045b79a12caab1e507258b84a2f00fa8c21eea85f
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [4]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.4 MB/s 


In [5]:
import os
import numpy as np
import pandas as pd
import csv
import os

In [6]:
os.environ["KAGGLE_USERNAME"] = "alessandromirone"
os.environ["KAGGLE_KEY"] = "c881e45ce076cb815820c1e582302936"

!kaggle datasets download bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows -f 0905_UkraineCombinedTweetsDeduped.csv.gzip

  0% 0.00/11.5M [00:00<?, ?B/s]
100% 11.5M/11.5M [00:00<00:00, 130MB/s]


In [7]:
#spark context
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
spark

In [8]:
#load zipped file in environment
from zipfile import ZipFile
with ZipFile('0905_UkraineCombinedTweetsDeduped.csv.gzip.zip', 'r') as zip:
    zip.extractall()
filename = r'0905_UkraineCombinedTweetsDeduped.csv.gzip'
df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
df

,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,...,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts
0,567289542,Saudi_Gazette,Saudi Gazette is a leading English language da...,Saudi Arabia,2,435540,134609,2012-04-30 13:01:28.000000,1566576845415501824,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 06:22:59.378541
1,1244796162751655936,RussianLadies1,Russia Ladies is one of the largest and most r...,Russia,903,412,554,2020-03-31 01:19:00.000000,1566576846745092097,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 07:15:37.978958
2,161871799,ZacatecasImagen,El periódico líder en Zacatecas,"Zacatecas, México",586,23190,178163,2010-07-02 00:48:16.000000,1566576846795411459,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 11:31:20.980226
3,984429894829592576,pulsoguayaco,"🌐✈Blog de aviación, viajes y economía para via...",🇪🇨🇺🇸,75,324,14459,2018-04-12 13:55:51.000000,1566576847051264002,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 11:31:20.967250
4,2445145434,Lyobserver,The Libya Observer is a key source for compreh...,Tripoli,96,77763,60461,2014-04-15 08:59:11.000000,1566576849035018240,2022-09-05 00:00:01,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 07:15:37.907896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44690,1532665429075935235,NicholasFreshne,"The views, information, or opinions expressed ...",USA,4,15,331,2022-06-03 10:08:29.000000,1566939152523972610,2022-09-05 23:59:40,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:40:18.561582
44691,1557539843219066882,Ludmila_Volkov,NaN,NaN,151,29,4241,2022-08-11 01:30:24.000000,1566939158182076417,2022-09-05 23:59:42,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:46:48.566650
44692,1565837550861828097,Tuchua2,#staywhitUkraine a small contribution to the e...,Leipzigngerd,89,579,1241,2022-09-02 23:02:57.000000,1566939194307612673,2022-09-05 23:59:50,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:46:48.556311
44693,31095945,shaancheema,I #Am #Zeeshan #Hussain #Equality #Justice (Be...,United Kingdom,114038,54654,193556,2009-04-14 10:34:44.000000,1566939215816007680,2022-09-05 23:59:56,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:46:48.546007


In [9]:
df.columns

Index(['userid', 'username', 'acctdesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreatedts', 'tweetid', 'tweetcreatedts',
       'retweetcount', 'text', 'hashtags', 'language', 'coordinates',
       'favorite_count', 'is_retweet', 'original_tweet_id',
       'original_tweet_userid', 'original_tweet_username',
       'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_screen_name', 'is_quote_status', 'quoted_status_id',
       'quoted_status_userid', 'quoted_status_username', 'extractedts'],
      dtype='object')

In [10]:
pd.unique(df.language) #many languages, only english texts will be considered

array(['en', 'es', 'ja', 'ar', 'no', 'uk', 'und', 'ru', 'zh', 'de', 'da',
       'it', 'vi', 'fr', 'pt', 'kn', 'ka', 'et', 'iw', 'in', 'lt', 'el',
       'fi', 'hi', 'is', 'th', 'nl', 'ca', 'eu', 'gu', 'pl', 'cy', 'tr',
       'bg', 'ro', 'lv', 'sv', 'ur', 'tl', 'sr', 'ta', 'ko', 'cs', 'ht',
       'fa', 'te', 'am', 'hu', 'bn', 'ml', 'mr', 'my', 'sl', 'ckb', 'ps',
       'or', 'pa', 'ne', 'hy'], dtype=object)

In [11]:
rdd = sc.parallelize(df.text[df.language == 'en'].sample(n=500,random_state=33))
rdd.take(50)

["“#Ukrainian President, Volodymyr #Zelenskyy has praised the work of the #Ukrainian Air Force for downing #Russian Kalibr cruise missiles, a helicopter and drones and vowed to do all to fully defend #Ukraine's skies from enemy missiles and aircraft.”https://t.co/kyZ8eGUGPp",
 'I SUPPORT UKRAINE 🇺🇦\n\nSTOP THE WAR\n\n#SlavaUkraini #StopRussia #StopPutin #PutinWarCriminal #NFT #NFTCommunity #LGBTQ #nftcollector #cryptoartist #NFTs #wezenaar #rickwezenaar https://t.co/sUu4PyKU9a',
 '@LPNH ooh, another #Pedoputin fondler\n\n#RussiaIsATerroristState #RussiaIsANaziState #NAFO',
 'Ukrainian military in Olhyne of #Kherson region https://t.co/hxroD3wmQQ https://t.co/hxroD3wmQQ https://t.co/8YYN3s6uza',
 'fuck a stock of laughing for Terrorist   #CIA #FBI #NATO #Cop #Cops #Police #Marines #Army #AirForce #Navy #News #BreakingNews #Worldnews #freemason #freemasons #Freemasonry',
 '#Russia to present next week in Geneva documented evidence of violations by US, #Ukraine of convention on bioweapons

### Preprocessing

In [12]:
import regex as re
from autocorrect import Speller
import unidecode
from string import punctuation

In [13]:
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemma = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = stopwords.words("english")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
def cleantxt(x):
    res = x.lower() #lowercase
    
    res = re.sub(r'<.*?>','',res) #remove html tags
   
    res = re.sub(r'https?://\S+|www\.\S+', '', res) #remove URLs
    
    res = re.sub(r'\d+', '', res) #remove numbers
   
    res = unidecode.unidecode(res) #convert accented characters to ascii characters
    
    emoji_char = re.compile("[" 
                               u"\U0001F600-\U0001F64F"  # emojis
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE) 
    res = emoji_char.sub(r'',res) #remove emojis, pictographs and other unusual characters

    res = re.sub("&amp", "and", res).replace("\n", " ") #remove refuses due to encoding 

    res = res.translate(str.maketrans('', '', punctuation)) #remove punctuation 

    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*') #remove stopwords
    res = pattern.sub('', res)

    res = re.sub('\s+'," ", res) #remove whitespaces

    spell_corrector = Speller(lang='en') #corrector NOTE: this is an english corrector: as a result, some slavic names may be changed
    res = spell_corrector(res)
               
    
    return res

    

    

In [15]:
rdd = rdd.map(cleantxt)
rdd.take(50)

['ukrainian president volodymyr zelenskyy praised work ukrainian air force downing russian caliber cruise missiles helicopter drones vowed fully defend ukraine skies enemy missiles aircraft',
 'support ukraine stop war slavaukraini stoprussia stopputin putinwarcriminal nft nftcommunity lgbtq nftcollector cryptoartist nts wezenaar rickwezenaar ',
 'link ooh another pedoputin founder russiaisaterroriststate russiaisanazistate nano',
 'ukrainian military ohne person region ',
 'fuck stock laughing terrorist cia fbi nato cop cops police marines army airforce navy news breakingnews worldnews freeman freemasonry freemasonry',
 'russia present next week geneva documented evidence violations us ukraine convention bioweapons russian ministry defense says',
 'beautiful mariupol putin justified russiaterroriststate visabanforrussians russia needs pay rebuild returned ukraine ',
 'oneillquigley ruedaungier jimfitzpatrick sure case never fought british empire world faced hitler naysayers said back 

In [16]:
rdd = rdd.map(lambda x: x.split(' ')) #"tokenization" (basket creation)
rdd.take(10)

[['ukrainian',
  'president',
  'volodymyr',
  'zelenskyy',
  'praised',
  'work',
  'ukrainian',
  'air',
  'force',
  'downing',
  'russian',
  'caliber',
  'cruise',
  'missiles',
  'helicopter',
  'drones',
  'vowed',
  'fully',
  'defend',
  'ukraine',
  'skies',
  'enemy',
  'missiles',
  'aircraft'],
 ['support',
  'ukraine',
  'stop',
  'war',
  'slavaukraini',
  'stoprussia',
  'stopputin',
  'putinwarcriminal',
  'nft',
  'nftcommunity',
  'lgbtq',
  'nftcollector',
  'cryptoartist',
  'nts',
  'wezenaar',
  'rickwezenaar',
  ''],
 ['link',
  'ooh',
  'another',
  'pedoputin',
  'founder',
  'russiaisaterroriststate',
  'russiaisanazistate',
  'nano'],
 ['ukrainian', 'military', 'ohne', 'person', 'region', ''],
 ['fuck',
  'stock',
  'laughing',
  'terrorist',
  'cia',
  'fbi',
  'nato',
  'cop',
  'cops',
  'police',
  'marines',
  'army',
  'airforce',
  'navy',
  'news',
  'breakingnews',
  'worldnews',
  'freeman',
  'freemasonry',
  'freemasonry'],
 ['russia',
  'present

In [17]:
def lem(res): #define lemmatization function (to be improved with POS)
    for i in range(len(res)):
        res[i] = lemma.lemmatize(res[i]) #lemmatize
    
    res = list(filter(None, res)) #remove NA elements still in the baskets
    return res

In [18]:
rdd = rdd.map(lem) #apply lemmatization
rdd.take(10)

[['ukrainian',
  'president',
  'volodymyr',
  'zelenskyy',
  'praised',
  'work',
  'ukrainian',
  'air',
  'force',
  'downing',
  'russian',
  'caliber',
  'cruise',
  'missile',
  'helicopter',
  'drone',
  'vowed',
  'fully',
  'defend',
  'ukraine',
  'sky',
  'enemy',
  'missile',
  'aircraft'],
 ['support',
  'ukraine',
  'stop',
  'war',
  'slavaukraini',
  'stoprussia',
  'stopputin',
  'putinwarcriminal',
  'nft',
  'nftcommunity',
  'lgbtq',
  'nftcollector',
  'cryptoartist',
  'nt',
  'wezenaar',
  'rickwezenaar'],
 ['link',
  'ooh',
  'another',
  'pedoputin',
  'founder',
  'russiaisaterroriststate',
  'russiaisanazistate',
  'nano'],
 ['ukrainian', 'military', 'ohne', 'person', 'region'],
 ['fuck',
  'stock',
  'laughing',
  'terrorist',
  'cia',
  'fbi',
  'nato',
  'cop',
  'cop',
  'police',
  'marine',
  'army',
  'airforce',
  'navy',
  'news',
  'breakingnews',
  'worldnews',
  'freeman',
  'freemasonry',
  'freemasonry'],
 ['russia',
  'present',
  'next',
  'we

### Algorithm 

1) description of the apriori algorithm \
2) implementation of apriori in spark \
actual code for implementations ( along with description of what's happening (point 2))

In [19]:
# a priori implementation; phase 1
# count frequency of each item (word) #WARNING: this function is extremly slow in a local setting

def count_freq(rdd):
    return (rdd.flatMap(lambda x: x)
            .map(lambda word: (word, 1)).
            reduceByKey(lambda a,b: a+b,numPartitions=16))
            

In [20]:
freq = count_freq(rdd) #all words frequencies
freq.take(10)

[('work', 13),
 ('stopputin', 7),
 ('nftcommunity', 6),
 ('cia', 1),
 ('yet', 5),
 ('avoided', 2),
 ('delivery', 4),
 ('shutdown', 6),
 ('g', 2),
 ('impose', 2)]

In [21]:
def threshold(x): 
    return True if x[1] > 50 else False #we define the threshold at 10% (50 for 500 tweets, 4400 for 44000)
tab = freq.filter(threshold)
result = tab
tab.take(10) #most frequent words and corresponding frequences

[('putin', 66),
 ('russia', 182),
 ('biden', 69),
 ('ukrainian', 59),
 ('russian', 139),
 ('ukraine', 214),
 ('war', 61)]

In [22]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [23]:
unique_words = freq.map(lambda x : x[0]) #rdd with all words in corpus
unique_words.take(10)

['work',
 'stopputin',
 'nftcommunity',
 'cia',
 'yet',
 'avoided',
 'delivery',
 'shutdown',
 'g',
 'impose']

In [26]:
c=2
while(tab.isEmpty() == False):

    tab_words = tab.map(lambda x : x[0]) #collect combinations of frequent words of recurrently created frequency tables
    combined = tab_words.cartesian(unique_words) #create all possible combinations with words in the table created in the previous iteration
   
    combined = combined.map(lambda item: removeReplica(item)) #remove replica: tuples of the form ((word1,word2),freq) are the
    #same as ((word2,word1),freq)
    combined = combined.filter(lambda item: len(item) == c) #retain only combinations of length c
    
    combined_2 = combined.cartesian(rdd) # build the confront table: here each of the combinations of frequent words are associated to each basket
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0])) #retain combinations if they appear in at least a basket
    
    
    combined_2 = combined_2.map(lambda item: item[0]) #unpack the list into combinations
    combined_2 = combined_2.map(lambda item: (item , 1)) #assign the counters to the combinations
    combined_2 = combined_2.reduceByKey(lambda a,b: a+b,numPartitions=16) #count the frequency of the combinations
    combined_2 = combined_2.filter(lambda item: item[1] >= 50) #retain only frequent combinations

    result = result.union(combined_2) #create the output table 
    
    tab = combined_2 #initialize the next table
    
    c = c+1 

TypeError: ignored

In [ ]:
combined_2 = combined.cartesian(rdd)
combined_2.take(10)

### Scalability 

how spark allow to scale the solution

### Results

Showcase of results and association rules retrieved from it

### conclusions

considerations about the results and further comments